In [15]:
import cohere
from cohere.classify import Example
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
from copy import deepcopy
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)


In [3]:
apiKey = ''
co = cohere.Client(apiKey)


semantic check

In [85]:
# Get the embeddings
def semantic_check(database, question_no, query):
    embeds = co.embed(texts=database[database.q_category==question_no][['answer1','answer2','answer3','answer4','answer5']].values.flatten().tolist(),
                      model='large',
                      truncate='LEFT').embeddings
                      
    # Create the search index, pass the size of embedding
    search_index = AnnoyIndex(len(embeds[0]), 'angular')
    # Add all the vectors to the search index
    for i in range(len(embeds)):
        search_index.add_item(i, embeds[i])
    search_index.build(10)  # 10 trees
    search_index.save(f'temp.ann')

    # process user input
    user_input_embeds = co.embed(texts=[query],
                                 model='large',
                                 truncate='LEFT').embeddings
    similar_item_ids = search_index.get_nns_by_vector(user_input_embeds[0], 10,
                                                      include_distances=True)
    # 1 is the question category
    # mean_distance = np.asarray(similar_item_ids[1]).mean()
    min_distance = min(similar_item_ids[1])
    return min_distance

Grammar Check

> generative check

In [82]:
def gen_grammar_check(query, trials = 3):
    similarity_values= []
    for i in range(trials):
        query = query.lower()
        response = co.generate(
            model='xlarge',
            prompt=f'This is a spell check generator that checks for grammar and corrects it. This also capitalizes the first letter of the sentence.\n\nSample: I would like a peice of pie.\nCorrect: I would like a piece of the pie.\n\nSample: my coworker said he used a financial planner to help choose his stocks so he wouldn\'t loose money.\nCorrect: My coworker said he used a financial planner to help him choose his stocks so he wouldn\'t lose money.\n\nSample: I ordered pizza, I also ordered garlic knots.\nCorrect: I ordered pizza; I also ordered garlic knots.\n\nSample: i bought winning lottery ticket the corner store\nCorrect: I bought my winning lottery ticket at the corner store.\n\nSample: try to reread your work to ensure you haven\'t left out any small words\nCorrect: Try to reread your work to ensure you haven\'t left out any small words.\n\nSample: I went to the movies with my sister. We will see the new comedy about dancing dogs.\nCorrect: I went to the movies with my sister. We saw the new comedy about dancing dogs.\n\nSample: the boy took their turn on the field.\nCorrect: The boy took his turn on the field.\n--\nSample: I could of won the race if I trained more.\nCorrect: I could have won the race if I had trained more.\n--\nSample: I went to the office, than i started my meeting.\nCorrect: I went to the office, then I started my meeting.\n--\nSample: {query}\nCorrect:',
            max_tokens=100,
            temperature=1.2,
            k=0,
            p=0.75,
            frequency_penalty=0,
            presence_penalty=0,
            stop_sequences=["--"],
            return_likelihoods='NONE')

        output = response.generations[0].text.rstrip("(\n)--")
        output = output.lstrip(" ")
        output = output.lower()

        embeds = co.embed(texts=[query, output],
                          model='large',
                          truncate='LEFT').embeddings

        sim_value = cosine_similarity(
            X=[embeds[0]], Y=[embeds[1]], dense_output=True)
        similarity_values.append(sim_value)
    similarity_score = max(similarity_values)
    print("gen:", similarity_values)
    return 1 if similarity_score > 0.997 else 0


> classification check

In [46]:
def class_grammar_check(query):
    response = co.classify(
        model='cdb39157-6b82-4cb4-92c5-9e6037623d79-ft',
        inputs=[f"{query}"])
    return(float(response.classifications[0].prediction))


Toxic behaviour check

> default model for toxic check

In [77]:
examples = [
    Example("you are hot trash", "Toxic"),
    Example("go to hell", "Toxic"),
    Example("get rekt moron", "Toxic"),
    Example("get a brain and use it", "Toxic"),
    Example("say what you mean, you jerk.", "Toxic"),
    Example("Are you really this stupid", "Toxic"),
    Example("I will honestly kill you", "Toxic"),
    Example("yo how are you", "Benign"),
    Example("I'm curious, how did that happen", "Benign"),
    Example("Try that again", "Benign"),
    Example("Hello everyone, excited to be here", "Benign"),
    Example("I think I saw it first", "Benign"),
    Example("That is an interesting point", "Benign"),
    Example("I love this", "Benign"),
    Example("We should try that sometime", "Benign"),
    Example("You should go for it", "Benign"),
    Example("people are not good.", "Benign")
]


def class_toxic_check(query):
    sentences = query.lower().rstrip('. ').split('.')
    for i in sentences:
        response = co.classify(
            model='large',
            inputs=[f"{i}"],
            examples=examples)
        if response.classifications[0].prediction == 'Toxic':
            return 1.0
    return 0.0


> custom model for toxic check

In [72]:
def custom_toxic_check(query):
    sentences = query.lower().rstrip('. ').split('.')
    for i in sentences:
        response = co.classify(
            model='8cec2377-0f7f-4557-81a4-7abc7dea3828-ft',
            inputs=[f"{i}"])
        if float(response.classifications[0].prediction) == 1.0:
            return 1.0
    return 0.0

Text Duplication Check

In [9]:
def Jaccard_Similarity(doc1, doc2): 
    
    if(doc1 == '' and doc2 == ''):
        return 0.0

    # List the unique words in a document
    words_doc1 = set(doc1.lower().split()) 
    words_doc2 = set(doc2.lower().split())
    
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)

    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)
        
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

def duplication_check(query):
    sentences = query.lower().rstrip('. ').split('.')
    similarities = []
    for i in range(len(sentences)):
        rest = deepcopy(sentences)
        rest.pop(i)
        rest = "".join(rest)
        score = Jaccard_Similarity(sentences[i], rest)
        similarities.append(score)
    duplication_ratio = sum(similarities)/(len(sentences)*0.08)

    if duplication_ratio>2.0:
        dup_score = 2
    elif duplication_ratio>1.0:
        dup_score = 1
    else:
        dup_score = 0
    return dup_score

#Testing

In [61]:
query_correct_answer = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. Long, narrow to wide trenches known as valley sinks or uvalas emerge when sink holes and dolines connect together due to slumping of materials along their margins or due to roof collapse of caves."

query_incorrect_answer = "During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Thus, pressure gradient from sea to land is created and the wind blows from the sea to the land which is known sea breeze. In the night the reversal of condition takes place. The land loses heat faster and is cooler than the sea. The pressure gradient is from the land to the sea. This breeze is known land breeze. Dumbass! During the day the land heats up faster and become warmer than the sea."

In [86]:
database=pd.read_csv("Book1.csv", delimiter=',')
database.head(1)

,q_category,question,answer1,answer2,answer3,answer4,answer5
0,1,Explain the evolution of valley sinks or uvalas.,"Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves, long, narrow to wide trenches called valley sinks or uvalas form.","Generally, the surface run-off simply goes down swallow and sink holes and flow as underground streams and re-emerge at a distance downstream through a cave opening. When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves, long, narrow to wide trenches called valley sinks or Uvalas form.","A sink hole is on opening more or less circular at the top and funnel-shaped towards the bottom with sizes varying in area from a few sq.m to a hectare and with depth from a less than half a metre to thirty metres or more. Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and re-emerge at a distance downstream through a cave opening. A doline is a collapsed sink. When sinkholes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves, long narrow to wide trenches called valley sinks or uvalas form.","Normally, the run-off goes down swallow and sink holes and drifts as underground streams and re-emerges at a distance downstream through a cave starting. When sink holes and dolines are merged due to slumping of substances alongside their margins or because of roof collapse of caves, trenches known as valley sinks or Uvalas are formed.","Valley sinks or uvulas are the same things. Valley sinks are erosional landforms of Karst topography. The following way valley sinks are evolved: In dolomite, limestone, or karst zones, the rocks are permeable and are made up of highly fused and broken rocks. In karst areas, small to medium-sized shallow pits are formed due to seepage of surface water, and these shallow pits are called sinkholes. Surface water from these sinkholes drains into the underground. The bottom of these sinkholes forms underground caves and sinkholes sometimes collapse and are commonly called collapse sinks or dolines. Typically, surface runoff runs down through sinkholes and flows underground, and at some distance through cave openings again to the surface. valley sinks or uvulas are developed when sinkholes and doline join together due to the collapse of cave ceilings."


In [63]:
def answer_analysis(database, question_no, query):
    semantic_check_score = semantic_check(database=database, question_no=question_no,query=query)
    gen_grammar_check_score = gen_grammar_check(query)
    class_grammar_check_score = class_grammar_check(query)
    class_toxic_check_score = class_toxic_check(query)
    custom_toxic_check_score = custom_toxic_check(query)
    duplication_check_score = duplication_check(query)
    print(f"semantic check score (less than 1 for correct answer): {semantic_check_score}")
    print(f"generative grammar check score (1 means correct): {gen_grammar_check_score}")
    print(f"class-based grammar check score (1 means correct): {class_grammar_check_score}")
    print(f"default model toxic check score (1 means toxic!): {class_toxic_check_score}")
    print(f"custom model toxic check score (1 means toxic!): {custom_toxic_check_score}")
    print(f"duplication check score (0=>no, 1=>detected, 2=>extreme)): {duplication_check_score}")


In [83]:
answer_analysis(database,1,query_correct_answer)

gen: [array([[1.]]), array([[0.72403635]]), array([[1.]])]
semantic check score (less than 1 for correct answer): 0.40302013158798217
generative grammar check score (1 means correct): 1
class-based grammar check score (1 means correct): 1.0
default model toxic check score (1 means toxic!): 0.0
custom model toxic check score (1 means toxic!): 0.0
duplication check score (0=>no, 1=>detected, 2=>extreme)): 0


In [84]:
answer_analysis(database,1,query_incorrect_answer)

gen: [array([[0.96575878]]), array([[0.95720239]]), array([[0.96834543]])]
semantic check score (less than 1 for correct answer): 1.357800054550171
generative grammar check score (1 means correct): 0
class-based grammar check score (1 means correct): 1.0
default model toxic check score (1 means toxic!): 1.0
custom model toxic check score (1 means toxic!): 1.0
duplication check score (0=>no, 1=>detected, 2=>extreme)): 1


In [88]:
query = "Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves, long, narrow to wide trenches called valley sinks or uvalas form."
semantic_check(database, 1, query) # complete answer (exact)

0.0015191452112048864

In [89]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. Long, narrow to wide trenches known as valley sinks or uvalas emerge when sink holes and dolines connect together due to slumping of materials along their margins or due to roof collapse of caves."
semantic_check(database, 1, query) # complete answer but paraphrased

0.2756789028644562

In [90]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. Valley sinks or uvalas arise when sink holes and dolines connect together owing to slumping of materials along their edges or ceiling collapse of caves."
semantic_check(database, 1, query) # complete answer but extremely paraphrased

0.33299142122268677

In [110]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. Valley sinks or uvalas arise when sink holes and dolines connect together owing to slumping of materials along their edges or ceiling collapse of caves. Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth."
semantic_check(database, 1, query) # complete answer but extremely paraphrased and dup

0.37606191635131836

In [111]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth."
semantic_check(database, 1, query) # complete answer but incomplete but extremely paraphrased 

0.6947792172431946

In [112]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth."
semantic_check(database, 1, query) # complete answer but incomplete but extremely paraphrased but dup

0.6006581783294678

In [114]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams."
semantic_check(database, 1, query) # very incomplete but extremely paraphrased

0.8300785422325134

In [115]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams. Surface run-off frequently just flows down swallow and sink holes as subterranean streams. Surface run-off frequently just flows down swallow and sink holes as subterranean streams."
semantic_check(database, 1, query) # very incomplete but extremely paraphrased but dup

0.7299461960792542

In [93]:
query = "When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves, long, narrow to wide trenches called valley sinks or uvalas form."
semantic_check(database, 1, query) # complete answer but incomplete (exact) 

0.46396487951278687

In [91]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening."
semantic_check(database, 1, query) # complete answer but incomplete (but paraphrased)

0.7044781446456909

In [96]:
query = "Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves, long, narrow to wide trenches called valley sinks or uvalas form. Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening."
semantic_check(database, 1, query) # complete answer (exact) and duplicated

0.25334981083869934

In [97]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. Long, narrow to wide trenches known as valley sinks or uvalas emerge when sink holes and dolines connect together due to slumping of materials along their margins or due to roof collapse of caves. The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening."
semantic_check(database, 1, query) # complete answer but paraphrased and duplicated

0.31706368923187256

In [94]:
query = "When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves, long, narrow to wide trenches called valley sinks or uvalas form. When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves, long, narrow to wide trenches called valley sinks or uvalas form."
semantic_check(database, 1, query) # complete answer but incomplete (exact) and duplicated

0.5305538773536682

In [95]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening."
semantic_check(database, 1, query) # complete answer but incomplete (but paraphrased) and duplicated

0.6648246645927429

In [99]:
query = "When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves."
semantic_check(database, 1, query) # very incomplete (exact)

0.755581796169281

In [109]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes."
semantic_check(database, 1, query) # very incomplete (but paraphrased)

0.8574945330619812

In [98]:
query = "When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves. When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves. When sink holes and dolines join together because of slumping of materials along their margins or due to roof collapse of caves."
semantic_check(database, 1, query) # very incomplete (exact) and extremely duplicated

0.7621968984603882

In [104]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes. The surface runoff frequently just disappears into sinkholes and swallow holes. The surface runoff frequently just disappears into sinkholes and swallow holes."
semantic_check(database, 1, query) # very incomplete (but paraphrased) and extremely duplicated

0.7983418703079224

In [116]:
query = "Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high."
semantic_check(database, 1, query) # complete answer (exact) + incorrect

0.8417791724205017

In [121]:
query = "Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. "
semantic_check(database, 1, query) # complete answer (exact) + incorrect (correct dup)

0.7815861105918884

In [122]:
query = "Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high."
semantic_check(database, 1, query) # complete answer (exact) + incorrect (incorrect dup)

0.9310697913169861

In [126]:
query = "Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Quite often the surface run-off simply goes down swallow and sink holes and flow as underground streams and reemerge at a distance downstream through a cave opening."
semantic_check(database, 1, query) # complete answer (exact) + incorrect

0.6873790621757507

In [127]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high."
semantic_check(database, 1, query) # complete answer but paraphrased + incorrect

0.8998052477836609

In [128]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening."
semantic_check(database, 1, query) # complete answer but paraphrased + incorrect + dup correct

0.7493038177490234

In [129]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening."
semantic_check(database, 1, query) # complete answer but paraphrased + incorrect + dup correct extreeme

0.7332528233528137

In [131]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high."
semantic_check(database, 1, query) # complete answer but paraphrased + incorrect + dup incorrect

1.000880241394043

In [132]:
query = "The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. The surface runoff frequently just disappears into sinkholes and swallow holes, where it flows as underground streams until emerging further downstream through a cave opening. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high."
semantic_check(database, 1, query) # complete answer but paraphrased + incorrect + dup both

0.7917125225067139

In [133]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high."
semantic_check(database, 1, query) # complete answer but extremely paraphrased + incorrect

0.8321602940559387

In [134]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth."
semantic_check(database, 1, query) # complete answer but extremely paraphrased + incorrect + correct dup

0.7118290066719055

In [135]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth."
semantic_check(database, 1, query) # complete answer but extremely paraphrased + incorrect + correct dup extreme

0.6846907138824463

In [136]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high."
semantic_check(database, 1, query) # complete answer but extremely paraphrased + incorrect + incorrect dup

0.8860904574394226

In [137]:
query = "Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Surface run-off frequently just flows down swallow and sink holes as subterranean streams, reemerging downstream through a cave mouth. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high."
semantic_check(database, 1, query) # complete answer but extremely paraphrased + incorrect

0.756028413772583

In [138]:
query = "During the day the land heats up faster and become warmer than the sea."
semantic_check(database, 1, query) # incorrect answer (small)

1.3008378744125366

In [139]:
query = "During the day the land heats up faster and become warmer than the sea. During the day the land heats up faster and become warmer than the sea."
semantic_check(database, 1, query) # incorrect answer (small) dup

1.2991032600402832

In [140]:
query = "During the day the land heats up faster and become warmer than the sea. During the day the land heats up faster and become warmer than the sea. During the day the land heats up faster and become warmer than the sea. During the day the land heats up faster and become warmer than the sea."
semantic_check(database, 1, query) # incorrect answer (small) ext dup

1.31532883644104

In [141]:
query = "During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Thus, pressure gradient from sea to land is created and the wind blows from the sea to the land which is known sea breeze. In the night the reversal of condition takes place. The land loses heat faster and is cooler than the sea. The pressure gradient is from the land to the sea. This breeze is known land breeze. Dumbass! During the day the land heats up faster and become warmer than the sea."
semantic_check(database, 1, query) # incorrect answer (large)

1.3217391967773438

In [142]:
query = "During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Thus, pressure gradient from sea to land is created and the wind blows from the sea to the land which is known sea breeze. In the night the reversal of condition takes place. The land loses heat faster and is cooler than the sea. The pressure gradient is from the land to the sea. This breeze is known land breeze. Dumbass! During the day the land heats up faster and become warmer than the sea. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. In the night the reversal of condition takes place. The land loses heat faster and is cooler than the sea."
semantic_check(database, 1, query) # incorrect answer (large) + dup

1.321215271949768

In [143]:
query = "During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Thus, pressure gradient from sea to land is created and the wind blows from the sea to the land which is known sea breeze. In the night the reversal of condition takes place. The land loses heat faster and is cooler than the sea. The pressure gradient is from the land to the sea. This breeze is known land breeze. Dumbass! During the day the land heats up faster and become warmer than the sea. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. In the night the reversal of condition takes place. The land loses heat faster and is cooler than the sea. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. Thus, pressure gradient from sea to land is created and the wind blows from the sea to the land which is known sea breeze. In the night the reversal of condition takes place. The land loses heat faster and is cooler than the sea. The pressure gradient is from the land to the sea. This breeze is known land breeze. Dumbass! During the day the land heats up faster and become warmer than the sea. During the day the land heats up faster and become warmer than the sea. Therfre, over the land the air rises giving rise to a low pressure area, whereas the sea is relatively cool and the pressure over sea it relatively high. In the night the reversal of condition takes place. The land loses heat faster and is cooler than the sea."
semantic_check(database, 1, query) # incorrect answer (large) + dup + ext dup

1.3188374042510986

In [144]:
query = " "
semantic_check(database, 1, query) # boundary case: space

1.4048672914505005

In [145]:
query = ""
semantic_check(database, 1, query) # boundary case: empty string

CohereError: invalid request: texts contains an element that is an empty string